In [153]:
import warnings
warnings.filterwarnings("ignore")

from carla.data.catalog import CsvCatalog
from carla import MLModelCatalog
from carla.recourse_methods import Clue, Wachter
from carla.models.negative_instances import predict_negative_instances
from carla.evaluation.benchmark import Benchmark
from sklearn.metrics import f1_score, accuracy_score
from sklearn.cluster import KMeans
from sklearn import metrics

import imageio
import timeit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import sys
sys.path.insert(0,'..')
from recourse_util import update_dataset, predict, print_scores 

num = 10
iter_id = 0

In [154]:
def train_model(dataset):
    training_params = {"lr": 0.005, "epochs": 4, "batch_size": 1, "hidden_size": [5]}

    model = MLModelCatalog(
        dataset,
        model_type="ann",
        load_online=False,
        backend="pytorch"
    )

    model.train(
        learning_rate=training_params["lr"],
        epochs=training_params["epochs"],
        batch_size=training_params["batch_size"],
        hidden_size=training_params["hidden_size"],
        force_train=True
    )
    
    return model

In [155]:
def load_dataset():
    dataset = CsvCatalog(
#         file_path='datasets/bimodal_dataset_1.csv',
#         file_path='datasets/unimodal_dataset_1.csv',
        file_path='datasets/unimodal_dataset_2.csv',
        categorical=[],
        continuous=['feature1', 'feature2'],
        immutables=[],
        target='target'
    )

    data_name = 'custom'
    return dataset

In [156]:
def train_recourse_method(method, model, dataset=None, data_name=None, hyperparams=None):
    rm = None
    if method == "clue":
        hyperparams = {
                "data_name": data_name,
                "train_vae": True,
                "width": 10,
                "depth": 3,
                "latent_dim": 12,
                "batch_size": 4,
                "epochs": 5,
                "lr": 0.0001,
                "early_stop": 20,
            }

        # load a recourse model and pass black box model
        rm = Clue(dataset, model, hyperparams)
        
    else:
        hyperparams = {
            "loss_type": "BCE",
            "t_max_min": 0.5/60
        }

        # load a recourse model and pass black box model
        rm = Wachter(model, hyperparams)
        
        
    return rm

In [157]:
def draw(data):

    plt.scatter(data['feature1'], data['feature2'], c=data['target'])
    plt.show()

In [158]:
def get_factuals(dataset, sample_num=5, max_m_iter=3):
    m_iter = 0
    model = train_model(dataset)
    factuals = predict_negative_instances(model, dataset._df)
    n_factuals = len(factuals)
    while (m_iter < max_m_iter and n_factuals < sample_num):
        model = train_model(dataset)
        factuals = predict_negative_instances(model, dataset._df)
        n_factuals = len(factuals)
        m_iter += 1
        
    return model, factuals

In [166]:
def execute_experiment_iteration(method, dataset, model, factuals, results, draw_state=False):
    print("Number of factuals", len(factuals))
    
#     add_data_statistics(model, dataset, results)
    
    if method == 'clue':
        rm = train_recourse_method('clue', model, dataset, data_name='custom')
    else:
        rm = train_recourse_method('wachter', model)
    
    start = timeit.default_timer()
    counterfactuals = rm.get_counterfactuals(factuals)
    stop = timeit.default_timer()
    print("Number of counterfactuals:", len(counterfactuals.dropna()))
    
    update_dataset(dataset, factuals, counterfactuals)
    
#     benchmark = CustomBenchmark(model, rm, factuals, counterfactuals, start - stop)
#     results['benchmark'] = benchmark.run_benchmark()
    
    add_data_statistics(model, dataset, results)
    
    if draw_state:
        draw(dataset._df)
    
    return dataset

In [160]:
results = {}
def get_empty_results():
    return {
        'datasets': [],
        'means': [],
        'covariances': [],
        'clustering': [],
        'accuracies': [],
        'f1_scores': [],
        'benchmark': []
    }

In [161]:
def add_data_statistics(model, dataset, results):
    results['datasets'].append(dataset._df.copy())
    results['means'].append(dataset._df[dataset.continuous].mean().to_numpy())
    results['covariances'].append(dataset._df[dataset.continuous].cov().to_numpy())
    results['clustering'].append(find_elbow(dataset))
    results['accuracies'].append(accuracy_score(np.array(dataset._df[dataset.target]), predict(model, dataset)))
    results['f1_scores'].append(f1_score(np.array(dataset._df[dataset.target]), predict(model, dataset)))

In [162]:
def find_elbow(dataset, n=10):
    ch_metrics = []
    x = dataset.df[dataset.continuous]
    
    for i in range(2, n):
        model = KMeans(n_clusters=i, random_state=1).fit(x)
        ch_metrics.append(metrics.calinski_harabasz_score(x, model.labels_))
        
    return ch_metrics.index(np.max(ch_metrics)) + 2

In [178]:
def generate_animation(results, method='clue'):
    data = results[method]['datasets']
    names = [f"images/{method}{str(n)}.png" for n in range(len(data))]
    
    for i, name in enumerate(names):
        for _ in range(1):
            plt.scatter(data[i]['feature1'], data[i]['feature2'], c=data[i]['target']) 
            
            plt.text(0, 0, "data_name", ha='left', va='center')     
            plt.text(1, 1, f"iteration {i+1}", ha='right', va='center')    
            plt.text(0, 1, "clue", ha='left', va='center')     
            plt.text(1, 0, f"2 samples", ha='right', va='center')

            plt.savefig(name)
            plt.close()
        
    gif_path = f"gifs/{method}_gif_{iter_id}.gif"
        
    with imageio.get_writer(f'{gif_path}', mode='I') as writer:
        for filename in names:
            image = imageio.imread(filename)
            writer.append_data(image)
            
    print(f"Saved gif to {gif_path}")
        
    for filename in set(names):
        os.remove(filename)

In [164]:
class CustomBenchmark(Benchmark):
    def __init__(
        self,
        mlmodel,
        recourse_method,
        factuals: pd.DataFrame,
        counterfactuals: pd.DataFrame,
        timer
    ) -> None:

        self._mlmodel = mlmodel
        self._recourse_method = recourse_method
        self._counterfactuals = counterfactuals.copy()
        self._timer = timer

        # Avoid using scaling and normalizing more than once
        if isinstance(mlmodel, MLModelCatalog):
            self._mlmodel.use_pipeline = False  # type: ignore

        self._factuals = factuals.copy()
    
#     def compute_ynn(self) -> pd.DataFrame:
#         return self.super().compute_ynn()
    
#     def compute_average_time(self) -> pd.DataFrame:
#         return self.super().compute_average_time()
    
#     def compute_distances(self) -> pd.DataFrame:
#         return self.super().compute_distances()
    
#     def compute_constraint_violation(self) -> pd.DataFrame:
#         return self.super().compute_constraint_violation()
    
#     def compute_redundancy(self) -> pd.DataFrame:
#         return self.super().compute_redundancy()
    
#     def compute_success_rate(self) -> pd.DataFrame:
#         return self.super().compute_success_rate()
    
#     def run_benchmark(self) -> pd.DataFrame:
#         return self.super().run_benchmark()

In [167]:
iter_id += 1
dataset = load_dataset()

clue_dataset = load_dataset()
clue_result = get_empty_results()
results['clue'] = clue_result

wachter_dataset = load_dataset()
wachter_result = get_empty_results()
results['wachter'] = wachter_result

iterations = 10
samples = 5

for i in range(iterations):
    clue_model, clue_factuals = get_factuals(clue_dataset, sample_num=samples)
    wachter_model, wachter_factuals = get_factuals(wachter_dataset, sample_num=samples)
    
    factuals = pd.merge(clue_factuals, wachter_factuals, how='inner', on=[*dataset.continuous, dataset.target])
    factuals = pd.merge(factuals, dataset._df, how='inner', on=dataset.continuous)
    
    if len(factuals) > samples:
        factuals = factuals.sample(samples)
    
    execute_experiment_iteration('clue', clue_dataset, clue_model, factuals, clue_result)
    execute_experiment_iteration('wachter', wachter_dataset, wachter_model, factuals, wachter_result)

balance on test set 0.5133333333333333, balance on test set 0.46
Epoch 0/3
----------
train Loss: 0.7193 Acc: 0.5133

test Loss: 0.6944 Acc: 0.4600

Epoch 1/3
----------
train Loss: 0.6819 Acc: 0.5600

test Loss: 0.6502 Acc: 1.0000

Epoch 2/3
----------
train Loss: 0.5971 Acc: 1.0000

test Loss: 0.5147 Acc: 1.0000

Epoch 3/3
----------
train Loss: 0.4550 Acc: 1.0000

test Loss: 0.3675 Acc: 1.0000

balance on test set 0.47333333333333333, balance on test set 0.58
Epoch 0/3
----------
train Loss: 0.4719 Acc: 0.8467

test Loss: 0.2280 Acc: 1.0000

Epoch 1/3
----------
train Loss: 0.1398 Acc: 1.0000

test Loss: 0.0703 Acc: 1.0000

Epoch 2/3
----------
train Loss: 0.0448 Acc: 1.0000

test Loss: 0.0246 Acc: 1.0000

Epoch 3/3
----------
train Loss: 0.0164 Acc: 1.0000

test Loss: 0.0092 Acc: 1.0000

Number of factuals 5
[INFO] 
Net: [utils.py __init__]
[INFO] VAE_gauss_net [fc_gauss_cat.py __init__]
[INFO] Total params: 0.00M [fc_gauss_cat.py create_net]
[INFO] 
Network: [train.py train_VAE]
[

[INFO] average time: 0.416590 seconds
 [train.py train_VAE]
[INFO] 
RESULTS: [train.py train_VAE]
[INFO] best_vlb_dev: -2.763392 [train.py train_VAE]
[INFO] best_vlb_train: -5.133230 [train.py train_VAE]
[INFO] nb_parameters: 1006 (1006.00B)
 [train.py train_VAE]
[INFO] 
Net: [utils.py __init__]
[INFO] VAE_gauss_net [fc_gauss_cat.py __init__]
[INFO] Total params: 0.00M [fc_gauss_cat.py create_net]
[INFO] Reading C:\Users\drobi\carla\models\autoencoders\clue\fc_VAE_custom_models\theta_best.dat
 [utils.py load]
[INFO] restoring epoch: 1, lr: 0.000100 [utils.py load]
Number of counterfactuals: 5
Number of factuals 5
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
Number of counterfactuals: 5


[INFO] init cost variables: [train.py train_VAE]
[INFO] it 0/5, vlb -5.488476,  [train.py train_VAE]
[INFO] time: 0.363000 seconds
 [train.py train_VAE]
[INFO] vlb -2.665203 (-inf)
 [train.py train_VAE]
[INFO] Writting C:\Users\drobi\carla\models\autoencoders\clue\fc_VAE_custom_models\theta_best.dat
 [utils.py save]
[INFO] it 1/5, vlb -5.342568,  [train.py train_VAE]
[INFO] time: 0.359000 seconds
 [train.py train_VAE]
[INFO] vlb -3.408636 (-2.665203)
 [train.py train_VAE]
[INFO] it 2/5, vlb -5.208235,  [train.py train_VAE]
[INFO] time: 0.377002 seconds
 [train.py train_VAE]
[INFO] vlb -5.450520 (-2.665203)
 [train.py train_VAE]
[INFO] it 3/5, vlb -5.117366,  [train.py train_VAE]
[INFO] time: 0.372000 seconds
 [train.py train_VAE]
[INFO] vlb -3.092664 (-2.665203)
 [train.py train_VAE]
[INFO] it 4/5, vlb -5.124391,  [train.py train_VAE]
[INFO] time: 0.370001 seconds
 [train.py train_VAE]
[INFO] vlb -3.671736 (-2.665203)
 [train.py train_VAE]
[INFO] Writting C:\Users\drobi\carla\models\au

Number of counterfactuals: 5
balance on test set 0.64, balance on test set 0.56
Epoch 0/3
----------
train Loss: 0.4599 Acc: 0.6667

test Loss: 0.3805 Acc: 0.8000

Epoch 1/3
----------
train Loss: 0.2837 Acc: 0.9267

test Loss: 0.2653 Acc: 1.0000

Epoch 2/3
----------
train Loss: 0.2085 Acc: 0.9933

test Loss: 0.2018 Acc: 0.9800

Epoch 3/3
----------
train Loss: 0.1642 Acc: 1.0000

test Loss: 0.1672 Acc: 1.0000

balance on test set 0.6466666666666666, balance on test set 0.7
Epoch 0/3
----------
train Loss: 0.5908 Acc: 0.6400

test Loss: 0.4748 Acc: 0.7000

Epoch 1/3
----------
train Loss: 0.4216 Acc: 0.7800

test Loss: 0.3085 Acc: 1.0000

Epoch 2/3
----------
train Loss: 0.2820 Acc: 0.9867

test Loss: 0.2031 Acc: 1.0000

Epoch 3/3
----------
train Loss: 0.1906 Acc: 1.0000

test Loss: 0.1432 Acc: 1.0000

Number of factuals 5
[INFO] 
Net: [utils.py __init__]
[INFO] VAE_gauss_net [fc_gauss_cat.py __init__]
[INFO] Total params: 0.00M [fc_gauss_cat.py create_net]
[INFO] 
Network: [train.py

In [168]:
results

{'clue': {'datasets': [     feature1  feature2  target
   0    0.080272  0.274545     0.0
   1    0.241592  0.065763     0.0
   2    0.096065  0.154848     0.0
   3    0.066525  0.280942     0.0
   4    0.195372  0.167314     0.0
   ..        ...       ...     ...
   195  0.885143  0.624946     1.0
   196  0.773658  0.672115     1.0
   197  0.826296  0.632205     1.0
   198  0.898391  0.558944     1.0
   199  0.782078  0.761226     1.0
   
   [200 rows x 3 columns],
        feature1  feature2  target
   0    0.080272  0.274545     0.0
   1    0.241592  0.065763     0.0
   2    0.096065  0.154848     0.0
   3    0.066525  0.280942     0.0
   4    0.195372  0.167314     0.0
   ..        ...       ...     ...
   195  0.885143  0.624946     1.0
   196  0.773658  0.672115     1.0
   197  0.826296  0.632205     1.0
   198  0.898391  0.558944     1.0
   199  0.782078  0.761226     1.0
   
   [200 rows x 3 columns],
        feature1  feature2  target
   0    0.080272  0.274545     0.0
   1    

In [179]:
generate_animation(results, 'clue')

Saved gif to gifs/clue_gif_2.gif
